<a href="https://colab.research.google.com/github/anuragsaraf1912/Babbar450/blob/master/getMarketData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output

In [94]:
def plotCandleStick(data):
  """Plots candlestick pattern using the Plotly library"""

  candlestick = go.Candlestick(
    x=data["Date"],
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    name="Price",
    increasing_line_color='green',
    decreasing_line_color='red',
    hovertext=[f"Returns: {round(row['Daily Returns'],2)}% <br> Volume: {round(row['Volume']/1000,2)}K <br> Open:{round(row['Open'],2)} <br> High:{round(row['High'],2)} <br> Low:{round(row['Low'],2)} <br> Close:{round(row['Close'],2)}" for index, row in data.iterrows()],
    hoverinfo="text"  # Display hovertext
  )

  # Create volume trace
  volume = go.Bar(
      x=data["Date"],
      y=data["Volume"],
      name="Volume",
      marker=dict(color='rgba(0, 0, 255, 0.3)')
  )

  # Create figure

  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(
      candlestick,
      secondary_y=False,
  )

  fig.add_trace(
      volume,
      secondary_y=True,
  )

  # Update layout
  fig.update_layout(
      title="Candlestick Chart with Volume",
      height = 1000,
      width = 2300,
      xaxis=dict(
          title="Date",
      ),
  )

  # Configure the y-axis for volume
  fig.update_yaxes(title_text="Price", secondary_y=False, )
  fig.update_yaxes(title_text="Volume", secondary_y=True, range =[0, 5*max(data["Volume"])])

  # Show figure
  fig.show()

In [88]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox

def getTicker(tick):
  if tick == '^NSEI':
    return '^NSEI'
  else:
    return tick + ".NS"

def on_value_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        startDate = start_date_picker.value
        endDate = end_date_picker.value
        ticker = getTicker(dropdown.value)
        if startDate and endDate and ticker:
          global df
          df = (yf.download(ticker, startDate, endDate)).reset_index()
          df['Daily Returns'] = 100*np.log(df['Adj Close']/df['Adj Close'].shift(1))
          df.dropna(inplace=True)
          clear_output(wait = True)
          display(widgets.HBox([dropdown, start_date_picker, end_date_picker])
)

# Date Picker Widgets
start_date_picker = widgets.DatePicker(description='Start Date:')
end_date_picker = widgets.DatePicker(description='End Date:')

# Text Entry Widget
options = ['^NSEI', 'HDFCBANK']

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=options,
    description='Select stock:'
)

# Observing changes
start_date_picker.observe(on_value_change)
end_date_picker.observe(on_value_change)
dropdown.observe(on_value_change)

# Displaying widgets
widgets.HBox([dropdown, start_date_picker, end_date_picker])
#Some stocks to look into: HDFCBANK, WSTCSTPAPR

In [95]:
plotCandleStick(df)

In [93]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns
1,2024-01-02,21751.349609,21755.599609,21555.650391,21665.800781,21665.800781,263700,-0.350628
2,2024-01-03,21661.099609,21677.000000,21500.349609,21517.349609,21517.349609,311900,-0.687545
3,2024-01-04,21605.800781,21685.650391,21564.550781,21658.599609,21658.599609,339200,0.654302
4,2024-01-05,21705.750000,21749.599609,21629.199219,21710.800781,21710.800781,309300,0.240728
5,2024-01-08,21747.599609,21763.949219,21492.900391,21513.000000,21513.000000,231500,-0.915247
...,...,...,...,...,...,...,...,...
179,2024-09-24,25921.449219,26011.550781,25886.849609,25940.400391,25940.400391,384100,0.005203
180,2024-09-25,25899.449219,26032.800781,25871.349609,26004.150391,26004.150391,278500,0.245454
181,2024-09-26,26005.400391,26250.900391,25998.400391,26216.050781,26216.050781,370900,0.811569
182,2024-09-27,26248.250000,26277.349609,26151.400391,26178.949219,26178.949219,490300,-0.141623


In [67]:
df['Daily Returns'].describe()

,Daily Returns
count,19.000000
mean,0.391694
std,0.812410
min,-1.742072
25%,0.091527
50%,0.299680
75%,0.721151
max,1.895871


In [68]:
# Create a histogram
fig = go.Figure(data=[go.Histogram(x=df['Daily Returns'])])

# Update layout for better appearance
fig.update_layout(
    title="Daily Returns Distribution",
    xaxis_title="Returns",
    yaxis_title="Count",
    bargap=0.1,  # Gap between bars
    bargroupgap=0.1,  # Gap between groups of bars
    height = 500,
    width = 1000
)

# Show the figure (interactive)
fig.show()

In [69]:
df.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns
10,2024-09-16,1667.000000,1677.800049,1664.050049,1670.949951,1670.949951,7586410,0.299680
11,2024-09-17,1678.000000,1678.000000,1664.150024,1668.800049,1668.800049,10063561,-0.128746
12,2024-09-18,1671.849976,1698.900024,1671.099976,1694.800049,1694.800049,20458671,1.545993
13,2024-09-19,1699.199951,1719.000000,1699.150024,1708.500000,1708.500000,15325888,0.805102
14,2024-09-20,1716.500000,1744.949951,1707.550049,1741.199951,1741.199951,30311693,1.895871
15,2024-09-23,1749.000000,1766.349976,1740.250000,1759.800049,1759.800049,13482675,1.062569
16,2024-09-24,1765.000000,1781.650024,1754.250000,1768.050049,1768.050049,16519518,0.467708
17,2024-09-25,1760.150024,1788.000000,1760.150024,1779.099976,1779.099976,15982656,0.623033
18,2024-09-26,1773.050049,1788.000000,1768.050049,1783.449951,1783.449951,15949822,0.244206
19,2024-09-27,1777.000000,1778.199951,1750.000000,1752.650024,1752.650024,17806751,-1.742072
